In [ ]:
# data_collection.ipynb

# This notebook demonstrates how to use the Twitter API and Kafka for real-time data collection.
# It streams tweets related to disaster events such as earthquakes, floods, hurricanes, etc.

# Required libraries
import tweepy
from kafka import KafkaProducer
import json

# Twitter API credentials
api_key = "YOUR_API_KEY"
api_secret = "YOUR_API_SECRET"
access_token = "YOUR_ACCESS_TOKEN"
access_token_secret = "YOUR_ACCESS_TOKEN_SECRET"

# Kafka Producer
producer = KafkaProducer(bootstrap_servers='localhost:9092', value_serializer=lambda v: json.dumps(v).encode('utf-8'))

# Twitter Stream Listener
class StreamListener(tweepy.StreamListener):
    def on_status(self, status):
        tweet = {
            'text': status.text,
            'user': status.user.screen_name,
            'created_at': str(status.created_at)
        }
        producer.send('disaster_tweets', tweet)

# Initialize the stream
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
stream = tweepy.Stream(auth, StreamListener())

# Start streaming tweets with specific keywords
stream.filter(track=['earthquake', 'flood', 'hurricane', 'fire'], languages=['en'])
